# 05 - Model Comparison
Train all three models and compare metrics/plots in one place.


In [ ]:
from pathlib import Path
import sys

ROOT = Path("..").resolve()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import numpy as np
import pandas as pd


In [ ]:
from src.models import make_model_registry, build_search
from src.eval import evaluate_models
from src.plots import plot_model_comparison
from _common import load_dataset, prepare_features
from src.split import SplitConfig

SEED = 42
TUNE_MODE = "off"  # off | fast | full
split_config = SplitConfig(test_rounds=6)
df, metadata = load_dataset()
train_df, val_df, trainval_df, test_df, features = prepare_features(df, metadata, split_config=split_config)

X_train = train_df[features]
y_train = train_df["LapTimeSeconds"].to_numpy()
X_val = val_df[features]
y_val = val_df["LapTimeSeconds"].to_numpy()

base_models = make_model_registry(features, random_state=SEED)
models = {k: build_search(k, v, random_state=SEED, mode=TUNE_MODE) for k, v in base_models.items()}
metrics, preds, fitted = evaluate_models(models, X_train, y_train, X_val, y_val)
metrics


In [ ]:
plot_model_comparison(metrics)
